## Preprocessing and cleaning 

In this notebook I explore the dataset and pre process it in order to generate a new dataset. In short the procedure contains
1) The creation of new columns  'Year', 'Month', 'Day', 'Hour', 'Minute', 'Currency Code',	'Rate',	'Amount Paid USD',	'Amount Received USD'
2) Standardise amount in different currencies to USD
 

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from data import *
df = get_data_local()
df = clean_data(df)
df = data_reduction(df)
df, df_test = train_test_split(df, test_size=0.3)

In [ ]:
df_test

In [ ]:
# Get number of rows
num_rows = len(df)
print("Number of rows: ", num_rows)

In [ ]:
df.shape

In [ ]:
# Column names and data type
df.dtypes

In [ ]:
#How many unique accounts there are
unique_count = df['from_account'].nunique()
print('Unique count of Account: ', unique_count)


In [ ]:
# Get the distribution of Is Laundering 
laundering_counts = df['is_laundering'].value_counts(normalize=True)
print(laundering_counts)

In [ ]:
df.columns

In [ ]:
# Identify Unique Currencies
#unique_currencies = df['currency_pair'].unique()
#print(unique_currencies)

In [ ]:
# Get the distribution the percentual distribution of flagged transactions by currency 

# Group by payment_currency and Is_Laundering, count the number of rows
#distribution = df.groupby(['currency_pair', 'is_laundering']).size().reset_index(name='Count')

# Calculate the total counts per currency and name it
#total_counts = df.groupby('currency_pair').size()
#total_counts.name = 'Count_total'

# Join the total_counts to the distribution dataframe
#distribution = distribution.set_index('currency_pair').join(total_counts)

# Calculate the percentage
#distribution['Percent'] = distribution['Count'] / distribution['Count_total'] * 100

# Reset the index
#distribution.reset_index(inplace=True)
#print(distribution)

In [ ]:
# Convert Timestamp into datetime
#df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Create new columns for year, month, day, hour and minute
#df['Year'] = df['Timestamp'].dt.year
#df['Month'] = df['Timestamp'].dt.month
#df['Day'] = df['Timestamp'].dt.day
#df['Hour'] = df['Timestamp'].dt.hour
#df['Minute'] = df['Timestamp'].dt.minute


In [ ]:
# Let's inspect the dataframe
df.head()

In [ ]:
# Now let's standardise the Amounts to US Dollar 

# First let' create a dictionary to map full currency names to their  ISO codes
#currency_codes = {
#    'Australian Dollar': 'AUD',
#    'Bitcoin': 'BTC',
#    'Brazil Real': 'BRL',
#    'Canadian Dollar': 'CAD',
#    'Euro': 'EUR',
#    'Mexican Peso': 'MXN',
#    'Ruble': 'RUB',
#    'Rupee': 'INR',
#    'Saudi Riyal': 'SAR',
#    'Shekel': 'ILS',
#    'Swiss Franc': 'CHF',
#    'UK Pound': 'GBP',
#    'Yen': 'JPY',
#    'Yuan': 'CNY',
#    'US Dollar': 'USD'
#}

# Replace the full currency names with their codes 
#df['Currency Code'] = df['Payment Currency'].replace(currency_codes)


In [ ]:
#import requests
# Get the exchange rate for all the currencies by connecting to Exchange Rate Data API

# Exchange rate data api key
#api_key = "wdiQOxfDckJMZr70O1Brmvlh56iJEfE7"

# Arbitrary date to fetch the exchange rates
#date = "2022-09-30"

# The list of currency codes to fetch the exchange rates
#currency_codes = ["GBP", "EUR", "AUD", "BTC", "BRL", "CAD", "MXN", "RUB", "INR", "SAR", "ILS", "CHF", "JPY", "CNY"]

# URL
#url = f"https://api.apilayer.com/exchangerates_data/{date}?symbols={','.join(currency_codes)}&base=USD"

# Define the headers
#headers = {
#  "apikey": api_key
#}

# Send a GET request to the API
#response = requests.get(url, headers=headers)

# Convert the response to JSON
#data = response.json()

# Create a DataFrame from the rates

#df_rates = pd.DataFrame(data['rates'].items(), columns=['Currency Code', 'Rate'])

# Display the DataFrame
#print(df_rates)


In [ ]:
# Merge df with df_rates on 'Currency Code', preserving all rows from df and filling in NaN for missing match
#df = df.merge(df_rates, on='Currency Code', how='left')

# Wherever the rate is NaN, that means the currency was USD. We can fill those with 1.
#df['Rate'] = df['Rate'].fillna(1)


In [ ]:
# Compute 'Amount Paid USD' and 'Amount Received USD'
#df['Amount Paid USD'] = df['Amount Paid'] * df['Rate']
#df['Amount Received USD'] = df['Amount Received'] * df['Rate']

In [ ]:
df

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
ohe.fit(df[['payment_format']])

df[ohe.get_feature_names_out()] = ohe.transform(df[['payment_format']])

ohe.fit(df[['receiving_currency']])
df[ohe.get_feature_names_out()] = ohe.transform(df[['receiving_currency']])

ohe.fit(df[['payment_currency']])
df[ohe.get_feature_names_out()] = ohe.transform(df[['payment_currency']])

In [ ]:
df.columns

In [ ]:
! pip install networkx
import networkx as nx
G = nx.MultiGraph()

# Add nodes to the graph for each unique card_id, merchant_name
G.add_nodes_from(df["from_account"].unique(), type='from_account')
G.add_nodes_from(df["to_account"].unique(), type='to_account')

In [ ]:
#len(df_temp["Account"].unique())

In [ ]:
#for i in df.columns: 
#    print(f"{i}" + " = " + f"{i}" +",")

In [ ]:
for _, row in df.iterrows():
    # Create a variable for each properties for each edge
    
        from_bank = row['from_bank'],
        from_account = row['from_account'],
        to_bank = row['to_bank'],
        to_account = row['to_account'],
        amount_paid_USD = row['amount_paid_USD'],
        amount_received_USD = row['amount_received_USD'],
        month = row['month'],
        day = row['day'],
        hour = row['hour'],
        minute = row['minute'],
        payment_format_ACH = row['payment_format_ACH'],
        payment_format_Bitcoin = row['payment_format_Bitcoin'],
        payment_format_Cash = row['payment_format_Cash'],
        payment_format_Cheque = row['payment_format_Cheque'],
        payment_format_Credit = row['payment_format_Credit Card'],
        receiving_currency_BRL = row['receiving_currency_BRL'],
        receiving_currency_CAD = row['receiving_currency_CAD'],
        receiving_currency_CHF = row['receiving_currency_CHF'],
        receiving_currency_CNY = row['receiving_currency_CNY'],
        receiving_currency_EUR = row['receiving_currency_EUR'],
        receiving_currency_GBP = row['receiving_currency_GBP'],
        receiving_currency_ILS = row['receiving_currency_ILS'],
        receiving_currency_INR = row['receiving_currency_INR'],
        receiving_currency_JPY = row['receiving_currency_JPY'],
        receiving_currency_MXN = row['receiving_currency_MXN'],
        receiving_currency_RUB = row['receiving_currency_RUB'],
        receiving_currency_SAR = row['receiving_currency_SAR'],
        receiving_currency_USD = row['receiving_currency_USD'],
        receiving_currency_XBT = row['receiving_currency_XBT'],
        receiving_currency_nan = row['receiving_currency_nan'],
        payment_currency_BRL = row['payment_currency_BRL'],
        payment_currency_CAD = row['payment_currency_CAD'],
        payment_currency_CHF = row['payment_currency_CHF'],
        payment_currency_CNY = row['payment_currency_CNY'],
        payment_currency_EUR = row['payment_currency_EUR'],
        payment_currency_GBP = row['payment_currency_GBP'],
        payment_currency_ILS = row['payment_currency_ILS'],
        payment_currency_INR = row['payment_currency_INR'],
        payment_currency_JPY = row['payment_currency_JPY'],
        payment_currency_MXN = row['payment_currency_MXN'],
        payment_currency_RUB = row['payment_currency_RUB'],
        payment_currency_SAR = row['payment_currency_SAR'],
        payment_currency_USD = row['payment_currency_USD'],
        payment_currency_XBT = row['payment_currency_XBT'],
        payment_currency_nan = row['payment_currency_nan']
 
        G.add_edge(row['from_account'], row['to_account'], 
            from_bank = from_bank,
            to_bank = to_bank,
            amount_paid_USD = amount_paid_USD,
            amount_received_USD = amount_received_USD,
            month = month,
            day = day,
            hour = hour,
            minute = minute,
            payment_format_ACH = payment_format_ACH,
            payment_format_Bitcoin = payment_format_Bitcoin,
            payment_format_Cash = payment_format_Cash,
            payment_format_Cheque = payment_format_Cheque,
            payment_format_Credit = payment_format_Credit,
            receiving_currency_BRL = receiving_currency_BRL,
            receiving_currency_CAD = receiving_currency_CAD,
            receiving_currency_CHF = receiving_currency_CHF,
            receiving_currency_CNY = receiving_currency_CNY,
            receiving_currency_EUR = receiving_currency_EUR,
            receiving_currency_GBP = receiving_currency_GBP,
            receiving_currency_ILS = receiving_currency_ILS,
            receiving_currency_INR = receiving_currency_INR,
            receiving_currency_JPY = receiving_currency_JPY,
            receiving_currency_MXN = receiving_currency_MXN,
            receiving_currency_RUB = receiving_currency_RUB,
            receiving_currency_SAR = receiving_currency_SAR,
            receiving_currency_USD = receiving_currency_USD,
            receiving_currency_XBT = receiving_currency_XBT,
            receiving_currency_nan = receiving_currency_nan,
            payment_currency_BRL = payment_currency_BRL,
            payment_currency_CAD = payment_currency_CAD,
            payment_currency_CHF = payment_currency_CHF,
            payment_currency_CNY = payment_currency_CNY,
            payment_currency_EUR = payment_currency_EUR,
            payment_currency_GBP = payment_currency_GBP,
            payment_currency_ILS = payment_currency_ILS,
            payment_currency_INR = payment_currency_INR,
            payment_currency_JPY = payment_currency_JPY,
            payment_currency_MXN = payment_currency_MXN,
            payment_currency_RUB = payment_currency_RUB,
            payment_currency_SAR = payment_currency_SAR,
            payment_currency_USD = payment_currency_USD,
            payment_currency_XBT = payment_currency_XBT,
            payment_currency_nan = payment_currency_nan)
  

In [ ]:
# Get the number of nodes and edges in the graph
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Print the number of nodes and edges
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

In [ ]:
G.size

In [ ]:
# Convert the graph to an adjacency matrix
adj_matrix = nx.adjacency_matrix(G).todense()

In [ ]:
adj_matrix

In [ ]:
# Get a small sample of the nodes in the graph
sample_nodes = list(G.nodes())[:10]

# Retrieve the properties of the sample nodes
node_properties = nx.get_node_attributes(G, 'type')

# Print the properties of the sample nodes
for node in sample_nodes:
    print(f"Node: {node}, Properties: {node_properties[node]}")

In [ ]:
sample_size = 5
for i, edge in enumerate(G.edges()):
    print(G.get_edge_data(*edge))
    if i >= sample_size - 1:
        break

In [ ]:
import collections
# Retrieve the properties errors of all the edges
edge_properties = nx.get_edge_attributes(G, 'errors')

# Count the number of edges by property value
edge_count_by_property = collections.Counter(edge_properties.values())

# Print the count of edges by property value
for property_value, count in edge_count_by_property.items():
    print(f"Property value: {property_value}, Count: {count}")

In [ ]:
# Prepare the data for input into the model
edge_list = list(G.edges(data=True))


In [ ]:
list(edge_list[i][2].values())

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class FraudGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FraudGNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x.squeeze(-1)

# Prepare the data for input into the model
edge_list = list(G.edges(data=True))
x = []
for edge in edge_list:
    edge_values = list(edge[2].values())
    print(edge_values)
    edge_values = [float(i[0]) if type(i) == tuple and type(i[0]) == str else i[0] if type(i) == tuple else i for i in edge_values]
    x.append(edge_values)
x = torch.tensor(x, dtype=torch.float)

In [ ]:
test = pd.DataFrame(x)
test.shape

In [ ]:
target = torch.tensor(df['is_laundering'].values, dtype=torch.float)

In [ ]:
x

In [ ]:
# Define the model
input_dim = len(x[0])
hidden_dim = 16
model = FraudGNN(input_dim, hidden_dim)
num_epochs=201

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(model.parameters())


In [ ]:
for i in range(num_epochs):
    # Forward pass
    output = model(x)
    # Compute the loss
    loss = criterion(output, target)
    if i % 20 == 0:
        print(f'Epoch: {i}, Loss: {loss.item()}')
    # Zero the gradients
    optimizer.zero_grad()
    # Perform backpropagation
    loss.backward()
    # Update the parameters
    optimizer.step()
    

In [ ]:
target

In [ ]:
from train import train_model

result = train_model
print(result)

In [ ]:
!pip install torchmetrics
import torch
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall, BinaryAUROC

In [ ]:
def evaluate_model(y_pred, y_test):
    """
    Evaluate trained model performance on the y_test
    """

    accuracy = BinaryAccuracy()

    precision = BinaryPrecision()

    recall = BinaryRecall()

    f1 = BinaryF1Score()

    auroc = BinaryAUROC()

    accuracy = accuracy(y_pred, y_test)

    precision = precision(y_pred, y_test)

    recall = recall(y_pred, y_test)

    f1 = f1(y_pred, y_test)

    auroc = auroc(y_pred, y_test)


    print(f"✅ Model evaluated")
    print(f"accuracy: {accuracy}")
    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")
    print(f"auroc: {auroc}")

    return (accuracy, precision, recall, f1, auroc)

In [ ]:
model.eval()

In [ ]:
test_data = x
test_data

In [ ]:
# Forward pass through the model to obtain predictions
with torch.no_grad():
    predictions = torch.sigmoid(model(test_data))  # Apply sigmoid to obtain probabilities

# Convert probabilities to binary predictions
threshold = 0.5  # Set the threshold for classification
binary_predictions = (predictions >= threshold).int()

# Print the binary predictions
print(binary_predictions)

In [ ]:
evaluate_model(target,binary_predictions)

In [ ]:
ohe = OneHotEncoder(sparse = False)
ohe.fit(df[['payment_format']])

df[ohe.get_feature_names_out()] = ohe.transform(df[['payment_format']])

ohe.fit(df[['receiving_currency']])
df[ohe.get_feature_names_out()] = ohe.transform(df[['receiving_currency']])

ohe.fit(df[['payment_currency']])
df[ohe.get_feature_names_out()] = ohe.transform(df[['payment_currency']])


! pip install networkx
import networkx as nx
G = nx.MultiGraph()

# Add nodes to the graph for each unique card_id, merchant_name
G.add_nodes_from(df["from_account"].unique(), type='from_account')
G.add_nodes_from(df["to_account"].unique(), type='to_account')

for _, row in df.iterrows():
    # Create a variable for each properties for each edge
    
        from_bank = row['from_bank'],
        from_account = row['from_account'],
        to_bank = row['to_bank'],
        to_account = row['to_account'],
        amount_paid_USD = row['amount_paid_USD'],
        amount_received_USD = row['amount_received_USD'],
        month = row['month'],
        day = row['day'],
        hour = row['hour'],
        minute = row['minute'],
        payment_format_ACH = row['payment_format_ACH'],
        payment_format_Bitcoin = row['payment_format_Bitcoin'],
        payment_format_Cash = row['payment_format_Cash'],
        payment_format_Cheque = row['payment_format_Cheque'],
        payment_format_Credit = row['payment_format_Credit Card'],
        receiving_currency_BRL = row['receiving_currency_BRL'],
        receiving_currency_CAD = row['receiving_currency_CAD'],
        receiving_currency_CHF = row['receiving_currency_CHF'],
        receiving_currency_CNY = row['receiving_currency_CNY'],
        receiving_currency_EUR = row['receiving_currency_EUR'],
        receiving_currency_GBP = row['receiving_currency_GBP'],
        receiving_currency_ILS = row['receiving_currency_ILS'],
        receiving_currency_INR = row['receiving_currency_INR'],
        receiving_currency_JPY = row['receiving_currency_JPY'],
        receiving_currency_MXN = row['receiving_currency_MXN'],
        receiving_currency_RUB = row['receiving_currency_RUB'],
        receiving_currency_SAR = row['receiving_currency_SAR'],
        receiving_currency_USD = row['receiving_currency_USD'],
        receiving_currency_XBT = row['receiving_currency_XBT'],
        receiving_currency_nan = row['receiving_currency_nan'],
        payment_currency_BRL = row['payment_currency_BRL'],
        payment_currency_CAD = row['payment_currency_CAD'],
        payment_currency_CHF = row['payment_currency_CHF'],
        payment_currency_CNY = row['payment_currency_CNY'],
        payment_currency_EUR = row['payment_currency_EUR'],
        payment_currency_GBP = row['payment_currency_GBP'],
        payment_currency_ILS = row['payment_currency_ILS'],
        payment_currency_INR = row['payment_currency_INR'],
        payment_currency_JPY = row['payment_currency_JPY'],
        payment_currency_MXN = row['payment_currency_MXN'],
        payment_currency_RUB = row['payment_currency_RUB'],
        payment_currency_SAR = row['payment_currency_SAR'],
        payment_currency_USD = row['payment_currency_USD'],
        payment_currency_XBT = row['payment_currency_XBT'],
        payment_currency_nan = row['payment_currency_nan']
 
        G.add_edge(row['from_account'], row['to_account'], 
            from_bank = from_bank,
            to_bank = to_bank,
            amount_paid_USD = amount_paid_USD,
            amount_received_USD = amount_received_USD,
            month = month,
            day = day,
            hour = hour,
            minute = minute,
            payment_format_ACH = payment_format_ACH,
            payment_format_Bitcoin = payment_format_Bitcoin,
            payment_format_Cash = payment_format_Cash,
            payment_format_Cheque = payment_format_Cheque,
            payment_format_Credit = payment_format_Credit,
            receiving_currency_BRL = receiving_currency_BRL,
            receiving_currency_CAD = receiving_currency_CAD,
            receiving_currency_CHF = receiving_currency_CHF,
            receiving_currency_CNY = receiving_currency_CNY,
            receiving_currency_EUR = receiving_currency_EUR,
            receiving_currency_GBP = receiving_currency_GBP,
            receiving_currency_ILS = receiving_currency_ILS,
            receiving_currency_INR = receiving_currency_INR,
            receiving_currency_JPY = receiving_currency_JPY,
            receiving_currency_MXN = receiving_currency_MXN,
            receiving_currency_RUB = receiving_currency_RUB,
            receiving_currency_SAR = receiving_currency_SAR,
            receiving_currency_USD = receiving_currency_USD,
            receiving_currency_XBT = receiving_currency_XBT,
            receiving_currency_nan = receiving_currency_nan,
            payment_currency_BRL = payment_currency_BRL,
            payment_currency_CAD = payment_currency_CAD,
            payment_currency_CHF = payment_currency_CHF,
            payment_currency_CNY = payment_currency_CNY,
            payment_currency_EUR = payment_currency_EUR,
            payment_currency_GBP = payment_currency_GBP,
            payment_currency_ILS = payment_currency_ILS,
            payment_currency_INR = payment_currency_INR,
            payment_currency_JPY = payment_currency_JPY,
            payment_currency_MXN = payment_currency_MXN,
            payment_currency_RUB = payment_currency_RUB,
            payment_currency_SAR = payment_currency_SAR,
            payment_currency_USD = payment_currency_USD,
            payment_currency_XBT = payment_currency_XBT,
            payment_currency_nan = payment_currency_nan)
        
        
adj_matrix = nx.adjacency_matrix(G).todense()

import collections
# Retrieve the properties errors of all the edges
edge_properties = nx.get_edge_attributes(G, 'errors')

# Count the number of edges by property value
edge_count_by_property = collections.Counter(edge_properties.values())

# Print the count of edges by property value
for property_value, count in edge_count_by_property.items():
    print(f"Property value: {property_value}, Count: {count}")
  
  

import torch
import torch.nn as nn
import torch.nn.functional as F

class FraudGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FraudGNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x.squeeze(-1)

# Prepare the data for input into the model
edge_list = list(G.edges(data=True))
x = []
for edge in edge_list:
    edge_values = list(edge[2].values())
    print(edge_values)
    edge_values = [float(i[0]) if type(i) == tuple and type(i[0]) == str else i[0] if type(i) == tuple else i for i in edge_values]
    x.append(edge_values)
x = torch.tensor(x, dtype=torch.float)


target = torch.tensor(df['is_laundering'].values, dtype=torch.float)

# Define the model
input_dim = len(x[0])
hidden_dim = 16
model = FraudGNN(input_dim, hidden_dim)
num_epochs=201

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(model.parameters())

for i in range(num_epochs):
    # Forward pass
    output = model(x)
    # Compute the loss
    loss = criterion(output, target)
    if i % 20 == 0:
        print(f'Epoch: {i}, Loss: {loss.item()}')
    # Zero the gradients
    optimizer.zero_grad()
    # Perform backpropagation
    loss.backward()
    # Update the parameters
    optimizer.step()
    
